# Merge 130k dataset w/ clean description

In [8]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 1.4 MB/s eta 0:00:00a 0:00:01


In [1]:
import pandas as pd
import numpy as np
import string
from sklearn.impute import KNNImputer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/augmoura/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/augmoura/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/augmoura/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/augmoura/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
df = pd.read_csv("../datasets/winemag-data-130k-v2.csv")
df_clean_description = pd.read_csv("../datasets/clean_description.csv")

In [3]:
df_clean_description = df_clean_description.drop(["Unnamed: 0"], axis=1)

In [4]:
df.index = df_clean_description.index

df = pd.concat([df, df_clean_description], axis=1)

# Add country names to missing countries

In [6]:
#Import of updated file (including missing countries & regions)
_ = pd.read_excel('../working_files/missing_countries.xlsx', index_col=0)

In [7]:
#
index_list = _.index.tolist()

#drop of lines to be updated in original file
df = df.drop(index=index_list)

#merge of original file (with removed prior lines) + new updated lines
df = pd.concat([df, _], axis=0)

#removing the repeated index column (0)
df = df.iloc[:, 1:]

# Clear nulls

In [9]:
#Replace the null of region_1,taster_name, taster_twitter_handle
df['region_1'] = df['region_1'].fillna('unknown')
df['taster_name'] = df['taster_name'].fillna('unknown')
df['taster_twitter_handle'] = df['taster_twitter_handle'].fillna('unknown')
df["designation"] = df["designation"].fillna('unknown')
df = df.drop(["region_2"], axis=1)

In [10]:
#Replace the null of price
imputer = KNNImputer(n_neighbors=5)
X = df[["points", "price"]]

# Fit and transform the prices
df[["points", "price"]] = imputer.fit_transform(X)

# Export initial data source

In [11]:
df.to_csv("../datasets/clean_dataset.csv")

# Perform LDA analysis in clean description

In [ ]:
cleaned_doc = df["clean_description_2"]

In [ ]:
cleaned_doc

In [ ]:
vectorizer = TfidfVectorizer(max_df = 0.5, min_df = 0.05)

vectorized_doc = vectorizer.fit_transform(cleaned_doc)
vectorized_doc = pd.DataFrame(
    vectorized_doc.toarray(),
    columns=vectorizer.get_feature_names_out()
)

vectorized_doc

In [ ]:
lda_model = LatentDirichletAllocation(n_components=5, max_iter=100)

lda_model.fit(vectorized_doc)

In [ ]:
topic_mixture = lda_model.transform(vectorized_doc)

In [ ]:
topics = pd.DataFrame(
    topic_mixture,
    columns=["topic_0", "topic_1", "topic_2", "topic_3", "topic_4"]
)

topics.head()

In [ ]:
def print_topics(lda_model, vectorizer, top_words):
    # 1. TOPIC MIXTURE OF WORDS FOR EACH TOPIC
    topic_mixture = pd.DataFrame(
        lda_model.components_,
        columns = vectorizer.get_feature_names_out()
    )
    
    # 2. FINDING THE TOP WORDS FOR EACH TOPIC
    ## Number of topics
    n_components = topic_mixture.shape[0]

    ## Top words for each topic
    for topic in range(n_components):
        print("-"*10)
        print(f"For topic {topic}, here are the top {top_words} words with weights:")

        topic_df = topic_mixture.iloc[topic]\
            .sort_values(ascending = False).head(top_words)
        
        print(round(topic_df,3))

print_topics(lda_model, vectorizer, 5)

# Concatenating df with topics

In [ ]:
df.index = topics.index

df = pd.concat([df, topics], axis=1)